# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [33]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

# 1) import any other libraries you might need
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc, asc, count, col, avg
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from pprint import pprint

In [27]:
# 2) instantiate a Spark session 
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [28]:
# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [29]:
# TODO: write your code to answer question 1
all_pages = user_log.select("page").dropDuplicates().sort("page").collect()
user_pages = user_log.select("page").where(user_log.userId == "").dropDuplicates().sort("page").collect()
unvisited_pages = set(all_pages) - set(user_pages)

pprint(list(unvisited_pages))

[Row(page='NextSong'),
 Row(page='Submit Downgrade'),
 Row(page='Error'),
 Row(page='Settings'),
 Row(page='Save Settings'),
 Row(page='Upgrade'),
 Row(page='Downgrade'),
 Row(page='Submit Upgrade'),
 Row(page='Logout')]


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [30]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select(["userId", "firstname", "lastname", "ts", "page", "level", "song"]) \
        .where(user_log.userId == "") \
        .sort("ts") \
        .collect()

[Row(userId='', firstname=None, lastname=None, ts=1513721196284, page='Login', level='free', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513721274284, page='Home', level='paid', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513722009284, page='Home', level='free', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513723183284, page='Home', level='paid', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513723184284, page='Login', level='paid', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513723587284, page='Login', level='free', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513724475284, page='Home', level='paid', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513724530284, page='Home', level='paid', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513728229284, page='Login', level='paid', song=None),
 Row(userId='', firstname=None, lastname=None, ts=1513729051

# Question 3

How many female users do we have in the data set?

In [31]:
# TODO: write your code to answer question 3
user_log.select(["gender", "userId"]) \
        .where(user_log.gender == "F") \
        .dropDuplicates() \
        .count()

462

# Question 4

How many songs were played from the most played artist?

In [32]:
# TODO: write your code to answer question 4
user_log.groupBy("artist") \
        .agg(count("artist")) \
        .orderBy(count("artist")) \
        .sort(desc("count(artist)")) \
        .show()

+--------------------+-------------+
|              artist|count(artist)|
+--------------------+-------------+
|            Coldplay|           83|
|       Kings Of Leon|           69|
|Florence + The Ma...|           52|
|            BjÃÂ¶rk|           46|
|       Dwight Yoakam|           45|
|       Justin Bieber|           43|
|      The Black Keys|           40|
|         OneRepublic|           37|
|                Muse|           36|
|        Jack Johnson|           36|
|           Radiohead|           31|
|        Taylor Swift|           29|
|Barry Tuckwell/Ac...|           28|
|          Lily Allen|           28|
|               Train|           28|
|           Daft Punk|           27|
|           Metallica|           27|
|          Nickelback|           27|
|          Kanye West|           26|
|Red Hot Chili Pep...|           24|
+--------------------+-------------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [35]:
# TODO: write your code to answer question 5
grp = user_log.where(user_log.page == "NextSong") \
              .groupBy("sessionId") \
              .agg(count("*").alias("count"))

grp.agg(avg(col("count"))).show()

+-----------------+
|       avg(count)|
+-----------------+
|6.830605564648118|
+-----------------+

